<a href="https://colab.research.google.com/github/LaJeremi/Tensorflow-nlp-tutorial-Practice-/blob/main/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/%2019_06_Combined_Topic_Models%2C_CTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 19-06 BERT 기반 복합 토픽 모델(Combined Topic Models, CTM)

출처: https://wikidocs.net/161310

# 복합 토픽 모델링(Combined Topic Modeling)
이 튜토리얼에서는 복합 토픽 모델(Combined Topic Model)을 사용하여 문서의 집합에서 토픽을 추출해보겠습니다.

# 토픽 모델(Topic Models)
토픽 모델을 사용하면 비지도 학습 방식으로 문서에 잠재된 토픽을 추출할 수 있습니다.

# 문맥을 반영한 토픽 모델(Contextualized Topic Models)
문맥을 반영한 토픽 모델(Contextualized Topic Models, CTM)이란 무엇일까요? CTM은 BERT 임베딩의 표현력과 토픽 모델의 비지도 학습의 능력을 결합하여 문서에서 주제를 가져오는 토픽 모델의 일종입니다.

# 파이썬 패키지(Python Package)
패키지는 여기를 참고하세요 링크.

https://github.com/MilaNLProc/contextualized-topic-models/actions https://pypi.python.org/pypi/contextualized_topic_models https://pepy.tech/badge/contextualized-topic-models


GPU를 사용하세요
우선, Colab에서 실습하기 전에 GPU 설정을 해주세요:

런타임 > 런타임 유형 변경을 클릭하세요.
노트 설정 > 하드웨어 가속기에서 'GPU'를 선택해주세요.
Reference

# Contextualized Topic Models, CTM 설치

In [ ]:

%%capture
!pip install contextualized-topic-models==2.2.0

In [ ]:

%%capture
!pip install pyldavis

# 노트북 재시작
원활한 실습을 위해서 노트북을 재시작 할 필요가 있습니다.

상단에서 런타임 > 런타임 재시작을 클릭해주세요.

# 2. 데이터 로드하기

# 데이터

학습을 위한 데이터가 필요합니다. 여기서는 하나의 라인(line)에 하나의 문서로 구성된 파일이 필요한데요. 우선, 여러분들의 데이터가 없다면 여기서 준비한 파일로 실습을 해봅시다.

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

In [ ]:
!head -n 1 dbpedia_sample_abstract_20k_unprep.txt


The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry


In [ ]:
!head -n 3 dbpedia_sample_abstract_20k_unprep.txt


The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and
Henry Howard, 13th Earl of Suffolk, 6th Earl of Berkshire (8 August 1779 – 10 August 1779) was a British peer, the son of Henry Howard, 12th Earl of Suffolk. His father died on 7 March 1779, leaving behind his pregnant widow. The Earldom of Suffolk became dormant until she


In [ ]:
!head -n 5 dbpedia_sample_abstract_20k_unprep.txt


The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and
Henry Howard, 13th Earl of Suffolk, 6th Earl of Berkshire (8 August 1779 – 10 August 1779) was a British peer, the son of Henry Howard, 12th Earl of Suffolk. His father died on 7 March 1779, leaving behind his pregnant widow. The Earldom of Suffolk became dormant until she
Marinko Matošević (Croatian pronunciation: [mariŋko matoʃeʋit͡ɕ]; born 8 August 1985) is an Aus

In [ ]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt" # EDIT THIS WITH THE FILE YOU UPLOAD


# 3. 전처리

# 필요한 것들을 임포트

여기서 전처리 된 텍스트를 사용하는 이유는 무엇일까요? Bag of Words를 구축하려면 특수문자가 없는 텍스트가 필요하고, 모든 단어를 사용하는 것보다는 빈번한 단어들만 사용하는 것이 좋습니다.

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

In [ ]:
nltk.download('stopwords')

documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# normalization 전처리 후 문서
preprocessed_documents[:2]

['mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry',
 'died march american photographer specialized photography operated studio silver spring maryland later lived florida magazine photographer year']

In [ ]:
# 전처리 전 문서 == documnets와 동일
unpreprocessed_corpus[:2]

['The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry',
 "Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and"]

In [ ]:
# 전체 단어 집합의 개수
print('bag of words에 사용 될 단어 집합의 개수 :',len(vocab))

bag of words에 사용 될 단어 집합의 개수 : 2000


In [ ]:
vocab[:5]


['among', 'character', 'earlier', 'factory', 'ancient']

전처리 되지 않은 문서는 문맥을 반영한 문서 임베딩을 얻기 위한 입력으로 사용할 것이기 때문에 제거해서는 안 됩니다.

전처리 전 문서와 전처리 후 문서를 TopicModelDataPreparation 객체에 넘겨줍니다. 이 객체는 bag of words와 문맥을 반영한 문서의 BERT 임베딩을 얻습니다. 여기서 사용할 pretrained BERT는 paraphrase-distilroberta-base-v1입니다.

In [ ]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v1")

training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

Batches:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
tp.vocab[:10]


['abbreviated',
 'academic',
 'academy',
 'access',
 'according',
 'achieved',
 'acquired',
 'acre',
 'acres',
 'across']

In [ ]:
len(tp.vocab)


2000

In [ ]:
set(vocab) == set(tp.vocab)


True

# 4. Combined TM 학습하기



In [ ]:

ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset) # run the model

Epoch: [20/20]	 Seen Samples: [400000/400000]	Train Loss: 135.5490373046875	Time: 0:00:20.533781: : 20it [07:26, 22.31s/it]


# 5. 결과 출력
# 토픽들

해당 메소드에는 각 토픽마다 몇 개의 단어를 보고 싶은지에 해당하는 파라미터를 넣어즐 수 있습니다. 여기서는 5개를 선택했습니다. 아래의 토픽들은 위키피디아(일반적인 주제)으로부터 얻은 토픽을 보여줍니다. 우리는 영어 문서로 학습하였으므로 각 토픽에 해당하는 단어들도 영어 단어들입니다.

In [ ]:
ctm.get_topic_lists(5)


[['game', 'series', 'first', 'developed', 'published'],
 ['school', 'high', 'states', 'united', 'new'],
 ['released', 'band', 'album', 'music', 'rock'],
 ['released', 'series', 'music', 'television', 'album'],
 ['built', 'house', 'historic', 'national', 'story'],
 ['film', 'directed', 'written', 'best', 'produced'],
 ['greek', 'modern', 'arabic', 'al', 'period'],
 ['world', 'born', 'summer', 'competed', 'silver'],
 ['south', 'north', 'west', 'mi', 'km'],
 ['company', 'founded', 'based', 'services', 'headquartered'],
 ['river', 'island', 'point', 'coast', 'named'],
 ['family', 'found', 'species', 'natural', 'mm'],
 ['journal', 'published', 'research', 'established', 'organization'],
 ['school', 'college', 'high', 'located', 'university'],
 ['american', 'born', 'served', 'former', 'football'],
 ['member', 'politician', 'elected', 'served', 'november'],
 ['built', 'church', 'building', 'house', 'historic'],
 ['island', 'range', 'islands', 'mountain', 'mount'],
 ['often', 'form', 'chemical

# 6. 시각화

# 시각화

우리의 토픽들을 시각화하기 위해서는 PyLDAvis를 사용합니다.



In [ ]:
 lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)


Sampling: [10/10]: : 10it [01:30,  9.10s/it]


In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

Sampling: [10/10]: : 10it [01:25,  8.56s/it]
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


# 7. 예측


## 이제 임의의 문서를 가져와서 어떤 토픽이 할당되었는지 확인할 수 있습니다. 예를 들어, 반도(peninsula)에 대한 첫번째 전처리 된 문서의 토픽을 예측해 봅시다.

In [ ]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) # get all the topic predictions


Sampling: [5/5]: : 5it [00:43,  8.74s/it]


In [ ]:

# 전처리 문서의 첫번째 문서
print(preprocessed_documents[0])
     

mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry


In [ ]:


import numpy as np
topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document

In [ ]:
ctm.get_topic_lists(5)[topic_number] #and the topic should be about natural location related things


['located', 'station', 'line', 'road', 'state']

# 8. 모델 저장 및 로드하기


# 차후 사용을 위해 모델 저장하기


In [ ]:
ctm.save(models_dir="./")


/usr/local/lib/python3.8/dist-packages/contextualized_topic_models/models/ctm.py:470: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn("This is an experimental feature that we has not been fully tested. Refer to the following issue:"


In [ ]:
# let's remove the trained model
del ctm

In [ ]:

ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, num_epochs=100, n_components=50)

ctm.load("/content/contextualized_topic_model_nc_50_tpm_0.0_tpv_0.98_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                      epoch=19)

/usr/local/lib/python3.8/dist-packages/contextualized_topic_models/models/ctm.py:495: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn("This is an experimental feature that we has not been fully tested. Refer to the following issue:"


In [ ]:
ctm.get_topic_lists(5)


[['game', 'series', 'first', 'developed', 'published'],
 ['school', 'high', 'states', 'united', 'new'],
 ['released', 'band', 'album', 'music', 'rock'],
 ['released', 'series', 'music', 'television', 'album'],
 ['built', 'house', 'historic', 'national', 'story'],
 ['film', 'directed', 'written', 'best', 'produced'],
 ['greek', 'modern', 'arabic', 'al', 'period'],
 ['world', 'born', 'summer', 'competed', 'silver'],
 ['south', 'north', 'west', 'mi', 'km'],
 ['company', 'founded', 'based', 'services', 'headquartered'],
 ['river', 'island', 'point', 'coast', 'named'],
 ['family', 'found', 'species', 'natural', 'mm'],
 ['journal', 'published', 'research', 'established', 'organization'],
 ['school', 'college', 'high', 'located', 'university'],
 ['american', 'born', 'served', 'former', 'football'],
 ['member', 'politician', 'elected', 'served', 'november'],
 ['built', 'church', 'building', 'house', 'historic'],
 ['island', 'range', 'islands', 'mountain', 'mount'],
 ['often', 'form', 'chemical